In [1]:
from math import sqrt

class Node:
    def __init__(self):
        self.mass = 0.0
        self.old_dx = 0.0
        self.old_dy = 0.0
        self.dx = 0.0
        self.dy = 0.0
        self.x = 0.0
        self.y = 0.0
        self.size = 0.


# This is not in the original java code, but it makes it easier to deal with edges
class Edge:
    def __init__(self):
        self.node1 = -1
        self.node2 = -1
        self.weight = 0.0

In [2]:
def linRepulsion(n1, n2, coefficient=0):
    xDist = n1.x - n2.x
    yDist = n1.y - n2.y
    
    distance = sqrt(xDist **2 + yDist**2) - (n1.size + n2.size)
    
    if distance > 0: # Clearly distance is always positive without collision detection
        factor = coefficient * n1.mass * n2.mass / distance**2
    elif distance < 0: # If the distance is smaller than the sum of radiuses then increase the repulsion
        factor = 100 * coefficient * n1.mass * n2.mass
        
    else: # If distance is 0 do nothing
        return
    # Apply the force
    n1.dx += xDist * factor
    n1.dy += yDist * factor
    n2.dx -= xDist * factor
    n2.dy -= yDist * factor

In [3]:
def linRepulsion_region(n, r, coefficient=0):
    xDist = n.x - r.massCenterX
    yDist = n.y - r.massCenterY
    distance2 = sqrt(xDist **2 + yDist**2)

    if distance2 > 0:
        factor = coefficient * n.mass * r.mass / distance2
        n.dx += xDist * factor
        n.dy += yDist * factor

In [4]:
def linAttraction(n1, n2, e, distributedAttraction, coefficient=0):
    xDist = n1.x - n2.x
    yDist = n1.y - n2.y
    distance = sqrt(xDist**2 + yDist**2) - n1.size - n2.size
    
    if distance > 0:
        if not distributedAttraction:
            factor = -coefficient * e * distance
        else:
            factor = -coefficient * e * distance / n1.mass
        n1.dx += xDist * factor
        n1.dy += yDist * factor
        n2.dx -= xDist * factor
        n2.dy -= yDist * factor

In [5]:
def linGravity(n, g):
    xDist = n.x
    yDist = n.y
    distance = sqrt(xDist **2 + yDist **2)

    if distance > 0:
        factor = n.mass * g / distance
        n.dx -= xDist * factor
        n.dy -= yDist * factor


In [6]:
def apply_repulsion(nodes, coefficient):
    i = 0
    for n1 in nodes:
        j = i
        for n2 in nodes:
            if j == 0:
                break
            linRepulsion(n1, n2, coefficient)
            j -= 1
        i += 1

In [7]:
def apply_gravity(nodes, gravity):
    for n in nodes:
        linGravity(n, gravity)

In [8]:
def apply_attraction(nodes, edges, distributedAttraction, coefficient, edgeWeightInfluence):
    # Optimization, since usually edgeWeightInfluence is 0 or 1, and pow is slow
    if edgeWeightInfluence == 0:
        for edge in edges:
            linAttraction(nodes[edge.node1], nodes[edge.node2], 1, distributedAttraction, coefficient)
    elif edgeWeightInfluence == 1:
        for edge in edges:
            linAttraction(nodes[edge.node1], nodes[edge.node2], edge.weight, distributedAttraction, coefficient)
    else:
        for edge in edges:
            linAttraction(nodes[edge.node1], nodes[edge.node2], pow(edge.weight, edgeWeightInfluence),
                          distributedAttraction, coefficient)


In [9]:
class Region:
    def __init__(self, nodes):
        self.mass = 0.0
        self.massCenterX = 0.0
        self.massCenterY = 0.0
        self.size = 0.0
        self.nodes = nodes
        self.subregions = []
        self.updateMassAndGeometry()

    def updateMassAndGeometry(self):
        if len(self.nodes) > 1:
            self.mass = 0
            massSumX = 0
            massSumY = 0
            for n in self.nodes:
                self.mass += n.mass
                massSumX += n.x * n.mass
                massSumY += n.y * n.mass
            self.massCenterX = massSumX / self.mass
            self.massCenterY = massSumY / self.mass

            self.size = 0.0
            for n in self.nodes:
                distance = sqrt((n.x - self.massCenterX) ** 2 + (n.y - self.massCenterY) ** 2)
                self.size = max(self.size, 2 * distance)

    def buildSubRegions(self):
        if len(self.nodes) > 1:

            leftNodes = []
            rightNodes = []
            for n in self.nodes:
                if n.x < self.massCenterX:
                    leftNodes.append(n)
                else:
                    rightNodes.append(n)

            topleftNodes = []
            bottomleftNodes = []
            for n in leftNodes:
                if n.y < self.massCenterY:
                    topleftNodes.append(n)
                else:
                    bottomleftNodes.append(n)

            toprightNodes = []
            bottomrightNodes = []
            for n in rightNodes:
                if n.y < self.massCenterY:
                    toprightNodes.append(n)
                else:
                    bottomrightNodes.append(n)

            if len(topleftNodes) > 0:
                if len(topleftNodes) < len(self.nodes):
                    subregion = Region(topleftNodes)
                    self.subregions.append(subregion)
                else:
                    for n in topleftNodes:
                        subregion = Region([n])
                        self.subregions.append(subregion)

            if len(bottomleftNodes) > 0:
                if len(bottomleftNodes) < len(self.nodes):
                    subregion = Region(bottomleftNodes)
                    self.subregions.append(subregion)
                else:
                    for n in bottomleftNodes:
                        subregion = Region([n])
                        self.subregions.append(subregion)

            if len(toprightNodes) > 0:
                if len(toprightNodes) < len(self.nodes):
                    subregion = Region(toprightNodes)
                    self.subregions.append(subregion)
                else:
                    for n in toprightNodes:
                        subregion = Region([n])
                        self.subregions.append(subregion)

            if len(bottomrightNodes) > 0:
                if len(bottomrightNodes) < len(self.nodes):
                    subregion = Region(bottomrightNodes)
                    self.subregions.append(subregion)
                else:
                    for n in bottomrightNodes:
                        subregion = Region([n])
                        self.subregions.append(subregion)

            for subregion in self.subregions:
                subregion.buildSubRegions()

    def applyForce(self, n, theta, coefficient=0):
        if len(self.nodes) < 2:
            linRepulsion(n, self.nodes[0], coefficient)
        else:
            distance = sqrt((n.x - self.massCenterX) ** 2 + (n.y - self.massCenterY) ** 2)
            if distance * theta > self.size:
                linRepulsion_region(n, self, coefficient)
            else:
                for subregion in self.subregions:
                    subregion.applyForce(n, theta, coefficient)

    def applyForceOnNodes(self, nodes, theta, coefficient=0): 
        for n in nodes:
            self.applyForce(n, theta, coefficient)


# Adjust speed and apply forces step
def adjustSpeedAndApplyForces(nodes, speed, speedEfficiency, jitterTolerance):
    # Auto adjust speed.
    totalSwinging = 0.0  # How much irregular movement
    totalEffectiveTraction = 0.0  # How much useful movement
    for n in nodes:
        swinging = sqrt((n.old_dx - n.dx) * (n.old_dx - n.dx) + (n.old_dy - n.dy) * (n.old_dy - n.dy))
        totalSwinging += n.mass * swinging
        totalEffectiveTraction += .5 * n.mass * sqrt(
            (n.old_dx + n.dx) * (n.old_dx + n.dx) + (n.old_dy + n.dy) * (n.old_dy + n.dy))

    # Optimize jitter tolerance.  The 'right' jitter tolerance for
    # this network. Bigger networks need more tolerance. Denser
    # networks need less tolerance. Totally empiric.
    estimatedOptimalJitterTolerance = .05 * sqrt(len(nodes))
    minJT = sqrt(estimatedOptimalJitterTolerance)
    maxJT = 10
    jt = jitterTolerance * max(minJT,
                               min(maxJT, estimatedOptimalJitterTolerance * totalEffectiveTraction / (
                                   len(nodes) * len(nodes))))

    minSpeedEfficiency = 0.05

    # Protective against erratic behavior
    if totalSwinging / totalEffectiveTraction > 2.0:
        if speedEfficiency > minSpeedEfficiency:
            speedEfficiency *= .5
        jt = max(jt, jitterTolerance)

    if totalSwinging == 0:
        targetSpeed = float('inf')
    else:
        targetSpeed = jt * speedEfficiency * totalEffectiveTraction / totalSwinging

    if totalSwinging > jt * totalEffectiveTraction:
        if speedEfficiency > minSpeedEfficiency:
            speedEfficiency *= .7
    elif speed < 1000:
        speedEfficiency *= 1.3

    # But the speed shoudn't rise too much too quickly, since it would
    # make the convergence drop dramatically.
    maxRise = .5
    speed = speed + min(targetSpeed - speed, maxRise * speed)

    # Apply forces.
    #
    # Need to add a case if adjustSizes ("prevent overlap") is
    # implemented.
    for n in nodes:
        swinging = n.mass * sqrt((n.old_dx - n.dx) * (n.old_dx - n.dx) + (n.old_dy - n.dy) * (n.old_dy - n.dy))
        factor = 0.1 * speed / (1.0 + sqrt(speed * swinging))
        df = sqrt(n.dx**2 + n.dy**2)
        factor = min(factor * df, 10.) / df
        n.x = n.x + (n.dx * factor)
        n.y = n.y + (n.dy * factor)

    values = {}
    values['speed'] = speed
    values['speedEfficiency'] = speedEfficiency

    return values


try:
    import cython

    if not cython.compiled:
        print("Warning: uncompiled fa2util module.  Compile with cython for a 10-100x speed boost.")
except:
    print("No cython detected.  Install cython and compile the fa2util module for a 10-100x speed boost.")

No cython detected.  Install cython and compile the fa2util module for a 10-100x speed boost.


In [10]:
import random
import time

import numpy
import scipy
from tqdm import tqdm


class Timer:
    def __init__(self, name="Timer"):
        self.name = name
        self.start_time = 0.0
        self.total_time = 0.0

    def start(self):
        self.start_time = time.time()

    def stop(self):
        self.total_time += (time.time() - self.start_time)

    def display(self):
        print(self.name, " took ", "%.2f" % self.total_time, " seconds")


class ForceAtlas2:
    def __init__(self,
                 # Behavior alternatives
                 outboundAttractionDistribution=False,  # Dissuade hubs
                 linLogMode=False, # Prevent overlap (NOT IMPLEMENTED)
                 edgeWeightInfluence=1.0,

                 # Performance
                 jitterTolerance=1.0,  # Tolerance
                 barnesHutOptimize=True,
                 barnesHutTheta=1.2,
                 multiThreaded=False,  # NOT IMPLEMENTED

                 # Tuning
                 scalingRatio=2.0,
                 gravity=1.0,

                 # Log
                 verbose=True):
        assert linLogMode == multiThreaded == False, "You selected a feature that has not been implemented yet..."
        self.outboundAttractionDistribution = outboundAttractionDistribution
        self.linLogMode = linLogMode
        self.edgeWeightInfluence = edgeWeightInfluence
        self.jitterTolerance = jitterTolerance
        self.barnesHutOptimize = barnesHutOptimize
        self.barnesHutTheta = barnesHutTheta
        self.scalingRatio = scalingRatio
        self.gravity = gravity
        self.verbose = verbose

    def init(self,
             G,  # a graph in 2D numpy ndarray format (or) scipy sparse matrix format
             pos=None,  # Array of initial positions
             sizes=None # Array of node sizes 
             ):
        isSparse = False
        if isinstance(G, numpy.ndarray):
            # Check our assumptions
            assert G.shape == (G.shape[0], G.shape[0]), "G is not 2D square"
            assert numpy.all(G.T == G), "G is not symmetric.  Currently only undirected graphs are supported"
            assert isinstance(pos, numpy.ndarray) or (pos is None), "Invalid node positions"
        elif scipy.sparse.issparse(G):
            # Check our assumptions
            assert G.shape == (G.shape[0], G.shape[0]), "G is not 2D square"
            assert isinstance(pos, numpy.ndarray) or (pos is None), "Invalid node positions"
            G = G.tolil()
            isSparse = True
        else:
            assert False, "G is not numpy ndarray or scipy sparse matrix"
            
        assert isinstance(sizes, numpy.ndarray) or (sizes is None), "Invalid node sizes"

        # Put nodes into a data structure we can understand
        nodes = []
        history = []
        for i in range(0, G.shape[0]):
            n = Node()
            if isSparse:
                n.mass = 1 + len(G.rows[i])
            else:
                n.mass = 1 + numpy.count_nonzero(G[i])
            n.old_dx = 0
            n.old_dy = 0
            n.dx = 0
            n.dy = 0
            if pos is None:
                n.x = random.random()
                n.y = random.random()
            else:
                n.x = pos[i][0]
                n.y = pos[i][1]

            if sizes is not None:
                n.size = sizes[i]
            nodes.append(n)

        # Put edges into a data structure we can understand
        edges = []
        es = numpy.asarray(G.nonzero()).T
        for e in es:  # Iterate through edges
            if e[1] <= e[0]: continue  # Avoid duplicate edges
            edge = Edge()
            edge.node1 = e[0]  # The index of the first node in `nodes`
            edge.node2 = e[1]  # The index of the second node in `nodes`
            edge.weight = G[tuple(e)]
            edges.append(edge)

        return nodes, edges

    # Given an adjacency matrix, this function computes the node positions
    # according to the ForceAtlas2 layout algorithm.  It takes the same
    # arguments that one would give to the ForceAtlas2 algorithm in Gephi.
    # Not all of them are implemented.  See below for a description of
    # each parameter and whether or not it has been implemented.
    #
    # This function will return a list of X-Y coordinate tuples, ordered
    # in the same way as the rows/columns in the input matrix.
    #
    # The only reason you would want to run this directly is if you don't
    # use networkx.  In this case, you'll likely need to convert the
    # output to a more usable format.  If you do use networkx, use the
    # "forceatlas2_networkx_layout" function below.
    #
    # Currently, only undirected graphs are supported so the adjacency matrix
    # should be symmetric.
    def forceatlas2(self,
                    G,  # a graph in 2D numpy ndarray format (or) scipy sparse matrix format
                    pos=None,  # Array of initial positions,
                    sizes=None,
                    iterations=100,  # Number of times to iterate the main loop
                    keep_history=False # Whether or not to return the historical values while fa2 is running
                    ):
        # Initializing, initAlgo()
        # ================================================================

        # speed and speedEfficiency describe a scaling factor of dx and dy
        # before x and y are adjusted.  These are modified as the
        # algorithm runs to help ensure convergence.
        speed = 1.0
        speedEfficiency = 1.0
        nodes, edges = self.init(G, pos,sizes)
        outboundAttCompensation = 1.0
        if self.outboundAttractionDistribution:
            outboundAttCompensation = numpy.mean([n.mass for n in nodes])
        # ================================================================

        # Main loop, i.e. goAlgo()
        # ================================================================

        barneshut_timer = Timer(name="BarnesHut Approximation")
        repulsion_timer = Timer(name="Repulsion forces")
        gravity_timer = Timer(name="Gravitational forces")
        attraction_timer = Timer(name="Attraction forces")
        applyforces_timer = Timer(name="AdjustSpeedAndApplyForces step")

        # Each iteration of this loop represents a call to goAlgo().
        niters = range(iterations)
        if self.verbose:
            niters = tqdm(niters)
        history = []
        for _i in niters:
            for n in nodes:
                n.old_dx = n.dx
                n.old_dy = n.dy
                n.dx = 0
                n.dy = 0

            # Barnes Hut optimization
            if self.barnesHutOptimize:
                barneshut_timer.start()
                rootRegion = Region(nodes)
                rootRegion.buildSubRegions()
                barneshut_timer.stop()

            # Charge repulsion forces
            repulsion_timer.start()
            # parallelization should be implemented here
            if self.barnesHutOptimize:
                rootRegion.applyForceOnNodes(nodes, self.barnesHutTheta, self.scalingRatio)
            else:
                apply_repulsion(nodes, self.scalingRatio)
            repulsion_timer.stop()

            # Gravitational forces
            gravity_timer.start()
            apply_gravity(nodes, self.gravity)
            gravity_timer.stop()

            # If other forms of attraction were implemented they would be selected here.
            attraction_timer.start()
            apply_attraction(nodes, edges, self.outboundAttractionDistribution, outboundAttCompensation,
                                     self.edgeWeightInfluence)
            attraction_timer.stop()

            # Adjust speeds and apply forces
            applyforces_timer.start()
            values = adjustSpeedAndApplyForces(nodes, speed, speedEfficiency, self.jitterTolerance)
            speed = values['speed']
            speedEfficiency = values['speedEfficiency']
            applyforces_timer.stop()

            # Add current positions to history
            if keep_history:
                positions = [(n.x, n.y) for n in nodes]
                history.append(positions)

        if self.verbose:
            if self.barnesHutOptimize:
                barneshut_timer.display()
            repulsion_timer.display()
            gravity_timer.display()
            attraction_timer.display()
            applyforces_timer.display()
        # ================================================================
        if not keep_history:
            return [(n.x, n.y) for n in nodes]
        else:
            return positions, history

    # A layout for NetworkX.
    #
    # This function returns a NetworkX layout, which is really just a
    # dictionary of node positions (2D X-Y tuples) indexed by the node name.
    def forceatlas2_networkx_layout(self, G, pos=None, sizes=None, iterations=100, keep_history=True):
        import networkx
        assert isinstance(G, networkx.classes.graph.Graph), "Not a networkx graph"
        assert isinstance(pos, dict) or (pos is None), "pos must be specified as a dictionary, as in networkx"
        assert isinstance(sizes, dict), "adjustSizes=True requires a sizes dict to be passed"
        M = networkx.to_scipy_sparse_matrix(G, dtype='f', format='lil')
        
        if sizes is not None:
            sizelist =  numpy.asarray([sizes[i] for i in G.nodes()])
        else:
            sizelist = None
        
        if pos is None:
            l = self.forceatlas2(M, pos=None, iterations=iterations, sizes=sizelist, keep_history=keep_history)
        else:
            poslist = numpy.asarray([pos[i] for i in G.nodes()])
            l = self.forceatlas2(M, pos=poslist, iterations=iterations, sizes=sizelist, keep_history=keep_history)
        if not keep_history:
            return dict(zip(G.nodes(), l))
        else:
            return [dict(zip(G.nodes(), i)) for i in l[1]]

In [11]:
forceatlas2 = ForceAtlas2(
                          # Behavior alternatives
                          outboundAttractionDistribution=True,  # Dissuade hubs
                          linLogMode=False,  # NOT IMPLEMENTED
                          edgeWeightInfluence=1.0,

                          # Performance
                          jitterTolerance=1.0,  # Tolerance
                          barnesHutOptimize=True,
                          barnesHutTheta=1.2,
                          multiThreaded=False,  # NOT IMPLEMENTED

                          # Tuning
                          scalingRatio=500.0,
                          gravity=1.0,

                          # Log
                          verbose=True)

In [12]:
import networkx as nx
new_graph = nx.readwrite.graphml.read_graphml('test.graphml')

In [13]:
import bezier
import networkx as nx
import numpy as np

def curved_edges(G, pos, dist_ratio=0.2, bezier_precision=20, polarity='random'):
    # Get nodes into np array
    edges = np.array(G.edges())
    l = edges.shape[0]

    if polarity == 'random':
        # Random polarity of curve
        rnd = np.where(np.random.randint(2, size=l)==0, -1, 1)
    else:
        # Create a fixed (hashed) polarity column in the case we use fixed polarity
        # This is useful, e.g., for animations
        rnd = np.where(np.mod(np.vectorize(hash)(edges[:,0])+np.vectorize(hash)(edges[:,1]),2)==0,-1,1)
    
    # Coordinates (x,y) of both nodes for each edge
    # e.g., https://stackoverflow.com/questions/16992713/translate-every-element-in-numpy-array-according-to-key
    # Note the np.vectorize method doesn't work for all node position dictionaries for some reason
    u, inv = np.unique(edges, return_inverse = True)
    coords = np.array([pos[x] for x in u])[inv].reshape([edges.shape[0], 2, edges.shape[1]])
    coords_node1 = coords[:,0,:]
    coords_node2 = coords[:,1,:]
    
    # Swap node1/node2 allocations to make sure the directionality works correctly
    should_swap = coords_node1[:,0] > coords_node2[:,0]
    coords_node1[should_swap], coords_node2[should_swap] = coords_node2[should_swap], coords_node1[should_swap]
    
    # Distance for control points
    dist = dist_ratio * np.sqrt(np.sum((coords_node1-coords_node2)**2, axis=1))

    # Gradients of line connecting node & perpendicular
    m1 = (coords_node2[:,1]-coords_node1[:,1])/(coords_node2[:,0]-coords_node1[:,0])
    m2 = -1/m1

    # Temporary points along the line which connects two nodes
    # e.g., https://math.stackexchange.com/questions/656500/given-a-point-slope-and-a-distance-along-that-slope-easily-find-a-second-p
    t1 = dist/np.sqrt(1+m1**2)
    v1 = np.array([np.ones(l),m1])
    coords_node1_displace = coords_node1 + (v1*t1).T
    coords_node2_displace = coords_node2 - (v1*t1).T

    # Control points, same distance but along perpendicular line
    # rnd gives the 'polarity' to determine which side of the line the curve should arc
    t2 = dist/np.sqrt(1+m2**2)
    v2 = np.array([np.ones(len(edges)),m2])
    coords_node1_ctrl = coords_node1_displace + (rnd*v2*t2).T
    coords_node2_ctrl = coords_node2_displace + (rnd*v2*t2).T

    # Combine all these four (x,y) columns into a 'node matrix'
    node_matrix = np.array([coords_node1, coords_node1_ctrl, coords_node2_ctrl, coords_node2])

    # Create the Bezier curves and store them in a list
    curveplots = []
    for i in range(l):
        nodes = node_matrix[:,i,:].T
        curveplots.append(bezier.Curve(nodes, degree=2).evaluate_multi(np.linspace(0,1,bezier_precision)).T)
      
    # Return an array of these curves
    curves = np.array(curveplots)
    return curves

In [29]:
import os
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np
import networkx as nx

def animate_fa2(G, f, num_iterations=100, output_dir='/tmp', edge_type='straight'):

    # Run FA2 and store the historical positions
    historical_positions = f.forceatlas2_networkx_layout(G,
                                                         pos=None,
                                                         sizes=nx.get_node_attributes(G,'degree'),
                                                         iterations=num_iterations,
                                                         keep_history=True)

    # Find the plot limits
    # This allows a fixed x,y axis so the animation doesn't continuously rescale
    all_positions = []
    for positions in historical_positions:
        for node_id in positions:
            all_positions.append(positions[node_id])
    all_positions = np.array(all_positions)
    xmin, ymin = all_positions.min(axis=0)
    xmax, ymax = all_positions.max(axis=0)
    
    # Plot
    num_zeros = len(str(num_iterations))
    i = 0
    plt.ioff() # Incase you're running in a notebook, this will stop it displaying n plots
    options = {
    'node_color': [node[1]['partition'] for node in G.nodes(data=True)], 
    'node_size': [node[1]['degree'] for node in G.nodes(data=True)],
    #'edge_color': [edge[2]['color'] for edge in nxG.edges(data=True)],
    'linewidths': 0.0,
    'width': 0.0
}
    
    for positions in historical_positions:
        plt.figure(i, figsize=(10,10))
        plt.gca().set_facecolor('k')
        
        # Draw straight line edges
        if edge_type == 'straight':
            nx.draw_networkx_edges(G, positions, width=1, **options , alpha=0.1)
        
        # Draw curved lines ala Gephi - see https://github.com/beyondbeneath/bezier-curved-edges-networkx
        else:
            curves = curved_edges(G, positions, polarity='fixed')
            lc = LineCollection(curves, color='w', alpha=0.1)
            plt.gca().add_collection(lc)
        
        # Draw nodes and finish up
        nx.draw_networkx_nodes(G, positions, **options)
        plt.tick_params(axis='both',which='both',bottom=False,left=False,labelbottom=False,labelleft=False)
        plt.xlim(xmin, xmax)
        plt.ylim(ymin, ymax)
        plot_filename = 'anim_frame_{}.png'.format(str(i).zfill(num_zeros))
        plt.savefig(os.path.join(output_dir, plot_filename), bbox_inches='tight')
        plt.close()

        # Increment plot counter
        i += 1

In [30]:
f = ForceAtlas2(
                          # Behavior alternatives
                          outboundAttractionDistribution=True,  # Dissuade hubs
                          linLogMode=False,  # NOT IMPLEMENTED
                          edgeWeightInfluence=1.0,

                          # Performance
                          jitterTolerance=1.0,  # Tolerance
                          barnesHutOptimize=True,
                          barnesHutTheta=1.2,
                          multiThreaded=False,  # NOT IMPLEMENTED

                          # Tuning
                          scalingRatio=500.0,
                          gravity=1.0,

                          # Log
                          verbose=True)

animate_fa2(new_graph, f, num_iterations=100, output_dir='fa2images/', edge_type='curved')


100%|██████████| 100/100 [02:57<00:00,  1.78s/it]


BarnesHut Approximation  took  14.62  seconds
Repulsion forces  took  156.14  seconds
Gravitational forces  took  0.83  seconds
Attraction forces  took  2.60  seconds
AdjustSpeedAndApplyForces step  took  2.48  seconds


In [31]:
import glob
import moviepy.editor as mpy
gif_name = 'fa2image'
fps = 12
file_list = glob.glob('fa2images/*.png') # Get all the pngs in the current directory
print(file_list)
list.sort(file_list, key=lambda x: int(x.split('_')[2].split('.png')[0])) # Sort the images by #, this may need to be tweaked for your use case
clip = mpy.ImageSequenceClip(file_list, fps=fps)
clip.write_gif('fa2images/{}.gif'.format(gif_name), fps=fps)

['fa2images/anim_frame_000.png', 'fa2images/anim_frame_001.png', 'fa2images/anim_frame_002.png', 'fa2images/anim_frame_003.png', 'fa2images/anim_frame_004.png', 'fa2images/anim_frame_005.png', 'fa2images/anim_frame_006.png', 'fa2images/anim_frame_007.png', 'fa2images/anim_frame_008.png', 'fa2images/anim_frame_009.png', 'fa2images/anim_frame_010.png', 'fa2images/anim_frame_011.png', 'fa2images/anim_frame_012.png', 'fa2images/anim_frame_013.png', 'fa2images/anim_frame_014.png', 'fa2images/anim_frame_015.png', 'fa2images/anim_frame_016.png', 'fa2images/anim_frame_017.png', 'fa2images/anim_frame_018.png', 'fa2images/anim_frame_019.png', 'fa2images/anim_frame_020.png', 'fa2images/anim_frame_021.png', 'fa2images/anim_frame_022.png', 'fa2images/anim_frame_023.png', 'fa2images/anim_frame_024.png', 'fa2images/anim_frame_025.png', 'fa2images/anim_frame_026.png', 'fa2images/anim_frame_027.png', 'fa2images/anim_frame_028.png', 'fa2images/anim_frame_029.png', 'fa2images/anim_frame_030.png', 'fa2ima

t:   4%|▍         | 4/100 [00:00<00:02, 37.35it/s, now=None]

MoviePy - Building file fa2images/fa2image.gif with imageio.



t: 100%|██████████| 100/100 [00:05<00:00, 13.08it/s, now=None]
                                                              

In [21]:
!pip install moviepy

     |████████████████████████████████| 378kB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 3.3MB 9.3MB/s eta 0:00:01
     |████████████████████████████████| 22.2MB 772kB/s  eta 0:00:01
     |████████████████████████████████| 143kB 13.2MB/s eta 0:00:01
     |████████████████████████████████| 61kB 8.3MB/s  eta 0:00:01
     |████████████████████████████████| 133kB 13.7MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 15.7MB/s eta 0:00:01
  Created wheel for moviepy: filename=moviepy-1.0.1-cp36-none-any.whl size=110786 sha256=a2f2abebe08e93fdcfe92934e634e7838ed47065780fde2b1444e82eba0f842c
  Stored in directory: /home/v-saflor/.cache/pip/wheels/a3/3c/07/45afe2bd5dbd3f935f445545d645f0f8c05d48340136367d45
  Created wheel for proglog: filename=proglog-0.1.9-cp36-none-any.whl size=6148 sha256=e9aa4727fff1e6df442975ce5c70da7cf3ecaf6bbe1cd666071ee23c5d7f3591
  Stored in directory: /home/v-saflor/.cache/p

In [ ]:
import matplotlib.pyplot as plt
nx.draw(new_graph, positions, **options, with_labels=False)
plt.show()